### Test of data instantiation

In [1]:
import torch
import numpy
from torch.utils.data import Dataset, DataLoader 
from torchvision.datasets import ImageFolder
import pandas as pd
from PIL import Image
import os


In [2]:

        
# img = torch.ones(6, 3)
# labels = [0, 1, 1, 1, 0, 0]

# dataset = ExpDataSet(img, labels)
# print(dataset[4])



In [3]:
# Local paths

label_path = "dataset/datasets/ringmerkingno/ringcodes.csv"

# Make sure to separate csv file on |
df = pd.read_csv(label_path, sep="|")

df.head()

,filename,code,color
0,LBNW(J22M)_20080310234907_79.jpg,J22M,NW
1,LBNW(J03Y)_20080311094828_79.jpg,J03Y,NW
2,LBBW(JR0T)_20080320193604_79.jpg,JR0T,BW
3,LBBW(J1EM)_20080321135128_79.jpg,J1EM,BW
4,LBBW(JZ3K)_20111027125715_79.jpg,JZ3K,BW


In [4]:

dl = df.sort_values("filename", ascending=True).reset_index(drop=True)
dl.head()
exp_labels = dl[["code", "color"]]
exp_labels.head()


,code,color
0,WT,BW
1,J0K8,WN
2,J0K8,WN
3,JA0H,WN
4,CN,NW


In [5]:
class ExpDataSet(Dataset):
    def __init__(self, img_path:str, labels_path:str, transform=None):
        self.img_path = img_path
        self.transform = transform

        # Retrieve labels and sort them in alphabetical order to match the images
        df = pd.read_csv(labels_path, sep="|")
        df = df.sort_values("filename", ascending=True).reset_index(drop=True)
        
        # Retrieve images from folder and match them with labels
        self.img_paths = df["filename"].apply(lambda e: os.path.join(img_path, e)).to_list()

        self.labels = list(df[["code","color"]].itertuples(index=False, name=None)) # Labels should be a list of tuples
        
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = Image.open(self.img_paths[idx]).convert('RGB')

        if self.transform: # Add transformation mask to image
            image = self.transform(image)
        
        return image, label

In [6]:
image_path = "dataset/datasets/ringmerkingno/images"

exp_dataset = ExpDataSet(img_path=image_path, labels_path=label_path)




In [8]:
print(exp_dataset[1])

(<PIL.Image.Image image mode=RGB size=1024x768 at 0x279826B0A50>, ('J0K8', 'WN'))
